In [2]:
from __future__ import print_function, division
import imgaug as ia
from imgaug import augmenters as iaa
import numpy as np
from numpy import array
from numpy import shape
from scipy import ndimage, misc
from skimage import data
import matplotlib.pyplot as plt
import six.moves as sm
import re
import os, glob
from datetime import datetime
from collections import defaultdict
from PIL import Image
try:
    from cStringIO import StringIO as BytesIO
except ImportError:
    from io import BytesIO
d = datetime.now()
np.random.seed(d.second)
ia.seed(d.second)

def main():
   draw_single_sequential_images()

def draw_single_sequential_images():
    # 원본 데이터 경로
    IMAGES_DIR = "/home/banana/image_augmentation/augmentation"
    # 경로의 폴더 리스트 추출
    grid_folder_list = array(os.listdir(IMAGES_DIR))
    # 경로의 폴더 경로 추출
    folders_root = glob.glob(IMAGES_DIR + "/*")
    # 하나의 폴더씩 처리
    for folder_list in range(len(grid_folder_list)):
        # 폴더 이름 저장
        FOLDER_NAME = grid_folder_list[folder_list]
        # 폴더 경로 저장
        FOLDER_ROOT = folders_root[folder_list]
        # 폴더에 있는 파일추출
        files_name = array(os.listdir(FOLDER_ROOT))
        # 폴더에 있는 그림파일 경로 추출
        files_root = glob.glob(IMAGES_DIR + "/" + FOLDER_NAME + "/*.png")
        # 하나의 파일씩 처리
        for file_list in range(len(files_name)) :
            # 파일 이름 저장
            FILE_NAME = files_name[file_list]
            # 파일 경로 저장
            FILES_ROOT = files_root[file_list]
            ia.seed(d.second)
            image = misc.imresize(ndimage.imread(FILES_ROOT)[0:96, 0:96], (96, 96))
            sometimes = lambda aug: iaa.Sometimes(0.5, aug)
            seq = iaa.Sequential(
                [
                    # apply the following augmenters to most images
                    iaa.Fliplr(0.5), # horizontally flip 50% of all images
                    iaa.Flipud(0.2), # vertically flip 20% of all images
                    # crop images by -5% to 10% of their height/width
                    sometimes(iaa.Affine(
                        # scale images to 80-120% of their size, individually per axis
                        scale={"x": (0.8, 1.2), "y": (0.8, 1.2)},
                        # translate by -20 to +20 percent (per axis)
                        rotate=(-45, 45), # rotate by -45 to +45 degrees
                        shear=(-16, 16), # shear by -16 to +16 degrees
                        order=[0, 1], # use nearest neighbour or bilinear interpolation (fast)
                        cval=(0, 255), # if mode is constant, use a cval between 0 and 255
                        # use any of scikit-image's warping modes (see 2nd image from the top for examples)
                    )),
                    # execute 0 to 5 of the following (less important) augmenters per image
                    # don't execute all of them, as that would often be way too strong
                    iaa.SomeOf((0, 5),
                        [
                            iaa.OneOf([
                                iaa.GaussianBlur((0, 3.0)), # blur images with a sigma between 0 and 3.0
                                iaa.AverageBlur(k=(2, 7)), # blur image using local means with kernel sizes between 2 and 7
                                iaa.MedianBlur(k=(3, 11)), # blur image using local medians with kernel sizes between 2 and 7
                            ]),
                            iaa.Sharpen(alpha=(0, 1.0), lightness=(0.75, 1.5)), # sharpen images
                            iaa.Emboss(alpha=(0, 1.0), strength=(0, 2.0)), # emboss images
                            # search either for all edges or for directed edges,
                            # blend the result with the original image using a blobby mask
                            iaa.AdditiveGaussianNoise(loc=0, scale=(0.0, 0.05*255), per_channel=0.5), # add gaussian noise to images
                            iaa.Invert(0.05, per_channel=True), # invert color channels
                            iaa.Add((-10, 10), per_channel=0.5), # change brightness of images (by -10 to 10 of original value)
                            iaa.AddToHueAndSaturation((-20, 20)), # change hue and saturation
                            # either change the brightness of the whole image (sometimes
                            # per channel) or change the brightness of subareas
                            iaa.OneOf([
                                iaa.Multiply((0.5, 1.5), per_channel=0.5),
                                iaa.FrequencyNoiseAlpha(
                                    exponent=(-4, 0),
                                    first=iaa.Multiply((0.5, 1.5), per_channel=True),
                                    second=iaa.ContrastNormalization((0.5, 2.0))
                                )
                            ]),
                            iaa.ContrastNormalization((0.5, 2.0), per_channel=0.5), # improve or worsen the contrast
                            iaa.Grayscale(alpha=(0.0, 1.0)),
                            sometimes(iaa.ElasticTransformation(alpha=(0.5, 1.0), sigma=0.2)), # move pixels locally around (with random strengths)
                            # sometimes move parts of the image around
                            sometimes(iaa.PiecewiseAffine(scale=(0.015, 0.045))), 
                            sometimes(iaa.PerspectiveTransform(scale=(0.01, 0.1)))
                        ],
                        random_order=True
                    )
                ],
                random_order=True
            )

            grid = seq.draw_grid(image, cols=8, rows=8)
            misc.imsave("./grid/" + FOLDER_NAME + "/" + FILE_NAME + ".jpg", grid)
    # 그리드 이미지 분할
    # 그리드 이미지 경로
    GRID_DIR = "/home/banana/image_augmentation/grid"
    GRID_AFTER_DIR = "/home/banana/image_augmentation/grid_after"
    # 경로의 폴더 리스트 추출
    grid_after_folder_list = array(os.listdir(GRID_DIR))
    print("Grid_folder_list ", grid_after_folder_list)
    # 경로의 폴더 경로 추출
    folders_root = glob.glob(GRID_DIR + "/*")
    print("folders_root ", folders_root)
    # 하나의 폴더씩 처리
    for folder_list in range(len(grid_after_folder_list)):
        # 폴더 이름 저장
        FOLDER_NAME = grid_folder_list[folder_list]
        print("FOLDER_NAME ", FOLDER_NAME)
        # 폴더 경로 저장
        FOLDER_ROOT = folders_root[folder_list]
        print("FOLDER_ROOT ", FOLDER_ROOT)
        # 폴더에 있는 파일추출
        files_name = array(os.listdir(FOLDER_ROOT))
        print("files_name ", files_name)
        # 폴더에 있는 그림파일 경로 추출
        files_root = glob.glob(GRID_DIR + "/" + FOLDER_NAME + "/*.jpg")
        print("files_root ", files_root)
        count = 0
        # 하나의 파일씩 처리
        for file_list in range(len(files_name)) :
            # 파일 이름 저장
            FILE_NAME = files_name[file_list]
            # 파일 경로 저장
            FILES_ROOT = files_root[file_list]
            img = Image.open(FILES_ROOT)
            img = img.convert("RGB")
            data = np.asarray(img)
            # 64 개의 사진으로 분할
            divide = []
            for slicey in range(0,768,96):
                for slicex in range(0,768,96):
                    divide.append(data[slicex:slicex+96,slicey:slicey+96,:])
            for data_list in range(len(divide)) :
                divide_img = Image.fromarray(divide[data_list],'RGB')
                divide_img.save(GRID_AFTER_DIR+"/"+FOLDER_NAME +"/"+
                                FOLDER_NAME+ str(count) + ".png")
                count = count + 1
            


class ExamplesImage(object):
    def __init__(self, image_height, image_width, title_cell_width, subtitle_height):
        self.rows = []
        self.image_height = image_height
        self.image_width = image_width
        self.title_cell_width = title_cell_width
        self.cell_height = image_height + subtitle_height
        self.cell_width = image_width

    def add_row(self, title, images, subtitles):
        assert len(images) == len(subtitles)
        images_rs = []
        for image in images:
            images_rs.append(ia.imresize_single_image(image, (self.image_height, self.image_width)))
        self.rows.append((title, images_rs, subtitles))

    def draw(self):
        rows_drawn = [self.draw_row(title, images, subtitles) for title, images, subtitles in self.rows]
        grid = np.vstack(rows_drawn)
        return grid

    def draw_row(self, title, images, subtitles):
        title_cell = np.zeros((self.cell_height, self.title_cell_width, 3), dtype=np.uint8) + 255
        title_cell = ia.draw_text(title_cell, x=2, y=12, text=title, color=[0, 0, 0], size=16)

        image_cells = []
        for image, subtitle in zip(images, subtitles):
            image_cell = np.zeros((self.cell_height, self.cell_width, 3), dtype=np.uint8) + 255
            image_cell[0:image.shape[0], 0:image.shape[1], :] = image
            image_cell = ia.draw_text(image_cell, x=2, y=image.shape[0]+2, text=subtitle, color=[0, 0, 0], size=11)
            image_cells.append(image_cell)

        row = np.hstack([title_cell] + image_cells)
        return row

#
# TODO this part is largely copied from generate_documentation_images, make DRY
#

def compress_to_jpg(image, quality=75):
    quality = quality if quality is not None else 75
    im = PIL.Image.fromarray(image)
    out = BytesIO()
    im.save(out, format="JPEG", quality=quality)
    jpg_string = out.getvalue()
    out.close()
    return jpg_string

def decompress_jpg(image_compressed):
    img_compressed_buffer = BytesIO()
    img_compressed_buffer.write(image_compressed)
    img = ndimage.imread(img_compressed_buffer, mode="RGB")
    img_compressed_buffer.close()
    return img

def arrdiff(arr1, arr2):
    nb_cells = np.prod(arr2.shape)
    d_avg = np.sum(np.power(np.abs(arr1.astype(np.float64) - arr2.astype(np.float64)), 2)) / nb_cells
    return d_avg

def save(fp, image, quality=75):
    image_jpg = compress_to_jpg(image, quality=quality)
    image_jpg_decompressed = decompress_jpg(image_jpg)

    # If the image file already exists and is (practically) identical,
    # then don't save it again to avoid polluting the repository with tons
    # of image updates.
    # Not that we have to compare here the results AFTER jpg compression
    # and then decompression. Otherwise we compare two images of which
    # image (1) has never been compressed while image (2) was compressed and
    # then decompressed.
    if os.path.isfile(fp):
        image_saved = ndimage.imread(fp, mode="RGB")
        #print("arrdiff", arrdiff(image_jpg_decompressed, image_saved))
        same_shape = (image_jpg_decompressed.shape == image_saved.shape)
        d_avg = arrdiff(image_jpg_decompressed, image_saved) if same_shape else -1
        if same_shape and d_avg <= 1.0: 
            print("[INFO] Did not save image '%s', because the already saved image is basically identical (d_avg=%.8f)" % (fp, d_avg,))
            return
        else:
            print("[INFO] Saving image '%s'..." % (fp,))

    with open(fp, "w") as f:
        f.write(image_jpg) 

if __name__ == "__main__":
    main()

/home/banana/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:51: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0.
Use ``matplotlib.pyplot.imread`` instead.
/home/banana/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:51: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
/home/banana/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:110: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.


Grid_folder_list  ['Hyun_soo' 'gill_hoon' 'jae_ho' 'junghyeok' 'hae_in' 'ho_ju' 'Seung_hyun'
 'yoondo' 'Jung_in' 'Kwang_ho']
folders_root  ['/home/banana/image_augmentation/grid/Hyun_soo', '/home/banana/image_augmentation/grid/gill_hoon', '/home/banana/image_augmentation/grid/jae_ho', '/home/banana/image_augmentation/grid/junghyeok', '/home/banana/image_augmentation/grid/hae_in', '/home/banana/image_augmentation/grid/ho_ju', '/home/banana/image_augmentation/grid/Seung_hyun', '/home/banana/image_augmentation/grid/yoondo', '/home/banana/image_augmentation/grid/Jung_in', '/home/banana/image_augmentation/grid/Kwang_ho']
FOLDER_NAME  Hyun_soo
FOLDER_ROOT  /home/banana/image_augmentation/grid/Hyun_soo
files_name  ['IMG_1556.CR2.png.jpg' 'IMG_3480.CR2.png.jpg' 'IMG_3567.CR2.png.jpg'
 'IMG_3661_view.png.jpg' 'IMG_2358.CR2.png.jpg' '1502967440827.png.jpg'
 'IMG_2404.CR2.png.jpg']
files_root  ['/home/banana/image_augmentation/grid/Hyun_soo/IMG_1556.CR2.png.jpg', '/home/banana/image_augmentation/